In [1]:
import folium

print(folium.__file__)
print(folium.__version__)

/home/filipe/.virtualenvs/iris/lib/python2.7/site-packages/folium/__init__.py
0.2.0.dev


In [2]:
import gpxpy
gpx = gpxpy.parse(open('/home/filipe/Dropbox/GPX/2014_08_05_farol.gpx'))

print("{} track(s)".format(len(gpx.tracks)))
track = gpx.tracks[0]

print("{} segment(s)".format(len(track.segments)))
segment = track.segments[0]

print("{} point(s)".format(len(segment.points)))

1 track(s)
1 segment(s)
1027 point(s)


In [3]:
data = []
segment_length = segment.length_3d()
for point_idx, point in enumerate(segment.points):
    data.append([point.longitude, point.latitude,
                 point.elevation, point.time, segment.get_speed(point_idx)])
    
from pandas import DataFrame

columns = ['Longitude', 'Latitude', 'Altitude', 'Time', 'Speed']
df = DataFrame(data, columns=columns)
df.head()

,Longitude,Latitude,Altitude,Time,Speed
0,-38.502595,-13.005390,10.9,2014-08-05 17:52:49.330,NaN
1,-38.502605,-13.005415,11.8,2014-08-05 17:52:49.770,2.672951
2,-38.502575,-13.005507,11.7,2014-08-05 17:52:54.730,3.059732
3,-38.502545,-13.005595,11.6,2014-08-05 17:52:57.750,4.220779
4,-38.502515,-13.005680,11.4,2014-08-05 17:53:00.720,3.939967


In [4]:
import numpy as np
import seawater as sw
from oceans.ff_tools import smoo1

_, angles = sw.dist(df['Latitude'], df['Longitude'])
angles = np.r_[0, np.deg2rad(angles)]

# Normalize the speed to use as the length of the arrows.
r = df['Speed'] / df['Speed'].max()
kw = dict(window_len=31, window='hanning')
df['u'] = smoo1(r * np.cos(angles), **kw)
df['v'] = smoo1(r * np.sin(angles), **kw)

In [5]:
import mplleaflet
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
df = df.dropna()

# This style was lost below.
ax.plot(df['Longitude'], df['Latitude'],
        color='darkorange', linewidth=5, alpha=0.5)

# This is preserved in the SVG icon.
sub = 10
kw = dict(color='deepskyblue', alpha=0.8, scale=10)
ax.quiver(df['Longitude'][::sub],
          df['Latitude'][::sub],
          df['u'][::sub],
          df['v'][::sub], **kw)

gj = mplleaflet.fig_to_geojson(fig=fig)

In [6]:
import folium

from glob import glob
import folium

lon, lat = -38.51386097, -13.00868051
zoom_start = 14

mapa = folium.Map(location=[lat, lon], tiles="Cartodb Positron",
                  zoom_start=zoom_start)

line_string = gj['features'][0]  # The first geometry is a lineString.
gjson = folium.features.GeoJson(line_string)

mapa.add_children(gjson)
mapa

In [7]:
line_string['properties']

{'color': '#FF8C00', 'opacity': 0.5, 'style': {}, 'weight': 5.0}

In [8]:
from IPython.display import HTML

HTML('<font color="{}">This should be darkorange!</font>'.format(line_string['properties']['color']))

In [9]:
mapa = folium.Map(location=[lat, lon], tiles="Cartodb Positron",
                  zoom_start=zoom_start)


icon_size = (14, 14)

for feature in gj['features']:
    if feature['geometry']['type'] == 'LineString':
        continue
    elif feature['geometry']['type'] == 'Point':
        lon, lat = feature['geometry']['coordinates']
        html = feature['properties']['html']
        
        icon_anchor = (feature['properties']['anchor_x'],
                       feature['properties']['anchor_y'])
        
        icon = folium.features.DivIcon(html=html,
                                       icon_size=(14, 14),
                                       icon_anchor=icon_anchor)
        marker = folium.map.Marker([lat, lon], icon=icon)
        mapa.add_children(marker)
    else:
        msg = "Unexpected geometry {}".format
        raise ValueError(msg(feature['geometry']))

mapa